In [1]:
import os 
import numpy as np 
import argparse
import sys
import datetime
import logging
from typing import Optional
from terrautils.spatial import scanalyzer_to_latlon, scanalyzer_to_utm
import json
import glob
import utm
import open3d as o3d
from math import pi, cos, sin
import multiprocessing

from terrautils.formats import create_geotiff, create_image
from terrautils.spatial import geojson_to_tuples

In [2]:
def get_bounding_box(meta, sensor):
    z_offset = 0.76
    scan_direction = int(meta['sensor_variable_metadata']['current setting Scan direction (automatically set at runtime)'])
    scan_distance = float(meta['sensor_variable_metadata']['current setting Scan distance (automatically set at runtime) [mm]'])/1000
    #scan_distance = float(meta['gantry_system_fixed_metadata']['system scan area e-w [m]']) #+ abs(west_loc_gantry_y - east_loc_gantry_y)
    fov_y = meta['sensor_fixed_metadata']['field of view y [m]']
    fov_x = scan_distance
    theta = np.radians(90)

    if sensor == 'east':
        # East
        east_loc_gantry_x = float(meta['sensor_fixed_metadata']['scanner east location in camera box x [m]'])
        east_loc_gantry_y = float(meta['sensor_fixed_metadata']['scanner east location in camera box y [m]'])
        east_loc_gantry_z = float(meta['sensor_fixed_metadata']['scanner east location in camera box z [m]'])

        gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]']) #+ east_loc_gantry_x
        gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]']) #+ east_loc_gantry_y
        gantry_z = float(meta['gantry_system_variable_metadata']['position z [m]']) + z_offset + east_loc_gantry_z#offset in m
        
        
        if scan_direction == 0:   
            gantry_y = gantry_y - scan_distance/2 + east_loc_gantry_y
            rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                           [np.sin(theta), np.cos(theta), 0],
                           [0, 0, 1]])


        elif scan_direction==1:
            gantry_y = gantry_y + scan_distance/2 + east_loc_gantry_y
            rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                            [np.sin(theta), np.cos(theta), 0],
                            [0, 0, 1]])
        bbox_center = scanalyzer_to_latlon(gantry_x, gantry_y)
            
    elif sensor == 'west':
        # West
        west_loc_gantry_x = float(meta['sensor_fixed_metadata']['scanner west location in camera box x [m]'])
        west_loc_gantry_y = float(meta['sensor_fixed_metadata']['scanner west location in camera box y [m]'])
        west_loc_gantry_z = float(meta['sensor_fixed_metadata']['scanner west location in camera box z [m]'])

        gantry_x = float(meta['gantry_system_variable_metadata']['position x [m]']) #+ west_loc_gantry_x
        gantry_y = float(meta['gantry_system_variable_metadata']['position y [m]']) #+ west_loc_gantry_y
        gantry_z = float(meta['gantry_system_variable_metadata']['position z [m]']) + z_offset + west_loc_gantry_z#offset in m
        
        
        if scan_direction == 0:   
            gantry_y = gantry_y - scan_distance/2 + west_loc_gantry_y
            rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                           [np.sin(theta), np.cos(theta), 0],
                           [0, 0, 1]])
            
        elif scan_direction==1:
            gantry_y = gantry_y + scan_distance/2 + west_loc_gantry_y
            rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                            [np.sin(theta), np.cos(theta), 0],
                            [0, 0, 1]])
        bbox_center = scanalyzer_to_latlon(gantry_x, gantry_y)

    B = gantry_z
    A_x = np.arctan((0.5*float(fov_x))/3.5)
    A_y = np.arctan((0.5*float(fov_y))/3.5)
    L_x = 2*B*np.tan(A_x)
    L_y = 2*B*np.tan(A_y)
    
    x_n = gantry_y + (L_x/2)
    x_s = gantry_y - (L_x/2)
    y_w = gantry_x + (L_y/2)
    y_e = gantry_x - (L_y/2)

    bbox_nw_latlon = scanalyzer_to_latlon(y_w, x_n)
    bbox_se_latlon = scanalyzer_to_latlon(y_e, x_s)
    
    lon_shift = 0.000020308287
    lat_shift = 0.000018292 #0.000015258894

    b_box =  (bbox_se_latlon[0] - lat_shift,
              bbox_nw_latlon[0] - lat_shift,
              bbox_nw_latlon[1] + lon_shift,
              bbox_se_latlon[1] + lon_shift)

    return b_box, bbox_center, rotation_matrix, gantry_z

In [3]:
def latlong_to_utm_bbox(b_box):
    l_w = (b_box[2], b_box[0])
    u_w = (b_box[2], b_box[1])
    u_e = (b_box[3], b_box[1])
    l_e = (b_box[3], b_box[0])

    l_w = utm.from_latlon(float(l_w[1]), float(l_w[0]))[:2]
    u_w = utm.from_latlon(float(u_w[1]), float(u_w[0]))[:2]
    u_e = utm.from_latlon(float(u_e[1]), float(u_e[0]))[:2]
    l_e = utm.from_latlon(float(l_e[1]), float(l_e[0]))[:2]

    return [l_w, u_w, u_e, l_e]

In [4]:
def latlong_to_utm_center(center):
    lat, long = center[0], center[1]
    center_utm = utm.from_latlon(lat, long)
    
    return center_utm[:2]

In [5]:
def get_corners(bbox):
    u_w = latlong_to_utm_bbox(bbox)[1]
    l_e = latlong_to_utm_bbox(bbox)[3]
    
    return u_w, l_e

In [6]:
def load_pcd(pcd_path, voxel_size=0.05):
    pcd = o3d.io.read_point_cloud(pcd_path)
    pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
    pcd_down.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    
    return pcd_down

In [7]:
def geo_reference_pcd(pcd, utm_x, utm_y, gantry_z, rotation_matrix):
    
    center_z = float(pcd.get_center()[2])
    corrected_pcd = pcd.translate([utm_x, utm_y, gantry_z], relative=False)
    rotated_pcd = corrected_pcd.rotate(rotation_matrix, center=corrected_pcd.get_center())

    #scaled_pcd = corrected_pcd.scale(0.000982699112208, corrected_pcd.get_center())

    #rotated_pcd = scaled_pcd.rotate(rotation_matrix, center=scaled_pcd.get_center())
    return rotated_pcd

In [8]:
def scale_pcd(pcd, scale_parameter):

    scaled_pcd = pcd.scale(float(scale_parameter), pcd.get_center())
    
    return scaled_pcd

In [9]:
def process_pcd(pcd):
    out_path = '/Volumes/Storage/3d_geo_correct_out/'
    os.makedirs(out_path) if not os.path.isdir(out_path) else None
    
    meta_path = glob.glob('/Volumes/Storage/3D/scanner3DTop/2020-02-01/2020-02-01__00-03-00-552/*_metadata.json')[0]
    with open(meta_path) as f:
        meta = json.load(f)['lemnatec_measurement_metadata']
    
    fname = os.path.splitext(os.path.basename(pcd))[-2]
    #print(['east' if 'east' in fname else 'west'])
    
    bbox, center, rot_matrix, gantry_z = get_bounding_box(meta, 'east' if 'east' in fname else 'west')
    u_w, l_e = get_corners(bbox)
    center = latlong_to_utm_center(center)
    real_scale = abs(u_w[0] - l_e[0])
    down_pcd = load_pcd(pcd)
    corr_pcd = geo_reference_pcd(down_pcd, center[0], center[1], gantry_z, rot_matrix)
    
    max_x, max_y, _ = corr_pcd.get_max_bound()
    min_x, min_y, _ = corr_pcd.get_min_bound()
    
    over_scale = abs(max_x - min_x)
    scale_parameter = real_scale/over_scale
    
    scale_pcd = scale_pcd(corr_pcd, scale_parameter)
    out_name = os.path.join(out_path, fname+'_corrected.ply')
    
    o3d.io.write_point_cloud(out_name, scale_pcd)
    
    return "Done"

In [22]:

pcd_paths = glob.glob('/Volumes/Storage/3D/scanner3DTop/2020-02-01/2020-02-01__00-03-00-552/*_0.ply')

for pcd in pcd_paths:
    process_pcd(pcd)
# with multiprocessing.Pool(2) as p:
#     p.map(process_pcd, pcd_paths)
#     #major_df = major_df.append(df)

# Single PCD processing

In [10]:
%%time
os.chdir('/home/emmanuelgonzalez/2020-01-15__00-00-33-078/')
meta_path = glob.glob('./*_metadata.json')[0]

# out_path = '/Volumes/Storage/3d_geo_correct_out/'
# os.makedirs(out_path) if not os.path.isdir(out_path) else None

with open(meta_path) as f:
    meta = json.load(f)['lemnatec_measurement_metadata']

for pcd in glob.glob('./*_0.ply'):
    fname = os.path.splitext(os.path.basename(pcd))[-2]
    print(['east' if 'east' in fname else 'west'])
    
    bbox, center, rot_matrix, gantry_z = get_bounding_box(meta, 'east' if 'east' in fname else 'west')
    u_w, l_e = get_corners(bbox)
    center_x = l_e[0] - ((l_e[0] - u_w[0])/2)
    center_y = u_w[1] - ((u_w[1] - l_e[1])/2)
    #print(f'{center_x}, {center_y}')
    print(u_w)
    print(l_e)
    print((center_x, center_y))

    #center = latlong_to_utm_center(center)
    # real_scale = abs(u_w[0] - l_e[0])
    # down_pcd = load_pcd(pcd)
    # corr_pcd = geo_reference_pcd(down_pcd, center[0], center[1], gantry_z, rot_matrix)
    
    # max_x, max_y, _ = corr_pcd.get_max_bound()
    # min_x, min_y, _ = corr_pcd.get_min_bound()
    
    # over_scale = abs(max_x - min_x)
    # scale_parameter = real_scale/over_scale
    
    # scale_pcd = scale_pcd(corr_pcd, scale_parameter)
    # out_name = os.path.join(out_path, fname+'_corrected.ply')
    
    # o3d.io.write_point_cloud(out_name, scale_pcd)
    
    
#     if 'east' in fname:
#         e_bbox, center, rot_matrix, gantry_z = get_bounding_box(meta, 'east')
#         u_w, l_e = get_corners(e_bbox)
#         center = latlong_to_utm_center(center)
#         down_pcd = load_pcd(pcd, 0.5)
#         corr_pcd = geo_reference_pcd(down_pcd, center[0], center[1], gantry_z)
        
        
#     elif 'west' in fname:
#         w_bbox, center, rot_matrix, gantry_z = get_bounding_box(meta, 'west')
#         u_w, l_e = get_corners(w_bbox)
#         center = latlong_to_utm_center(center)
#         down_pcd = load_pcd(pcd, 0.5)
#         corr_pcd = geo_reference_pcd(down_pcd, center[0], center[1], gantry_z)

['west']
(408989.9043813888, 3660064.109268342)
(409012.2155873272, 3660063.123353666)
(409001.059984358, 3660063.6163110044)
['east']
(408992.3209947557, 3660064.090392829)
(409014.63220069284, 3660063.1044781543)
(409003.47659772425, 3660063.5974354916)
CPU times: user 21.5 ms, sys: 674 µs, total: 22.2 ms
Wall time: 17.6 ms
